In [ ]:
import ast
import math
import random
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from collections import defaultdict
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Pt
import sys

sys.path.append("..")
from src.config import LAYOUT_DATA_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR

LAYOUT_PPTX = LAYOUT_DATA_DIR / "layout.pptx"
OUTPUT_LAYOUT_DIR = LAYOUT_DATA_DIR / "output"
LAYOUT_CSV = LAYOUT_DATA_DIR / "layout.csv"

ASSOC_RULES_CSV = PROCESSED_DATA_DIR / "association_rules_pyspark.csv"
TRANSACTIONS_CSV = RAW_DATA_DIR / "transactions.csv"
SKU_CSV = RAW_DATA_DIR / "sku.csv"  # cần cột: Sku, SDeptName

OUTPUT_LAYOUT_CSV = OUTPUT_LAYOUT_DIR / "layout_new.csv"
OUTPUT_PREVIEW_PNG = OUTPUT_LAYOUT_DIR / "layout_new.png"
OUTPUT_PPTX = OUTPUT_LAYOUT_DIR / "layout_new.pptx"

CATEGORY_COL = "SDeptName"
RANDOM_SEED = 42

# Heuristic mục tiêu
ALPHA_PAIR = 1.0  # kéo các cặp (lift/conf) lại gần
BETA_ENTRANCE = 1.0  # kéo support cao gần Entrance
GAMMA_SUPPORT = 0.7  # làm dịu outlier support

# Simulated Annealing
SA_ITERS = 8000
SA_START_TEMP = 1.0
SA_END_TEMP = 0.01

# Vẽ preview
PADDING_RATIO = 0.06

# Nhận diện keyword
ENTRANCE_KEYWORDS = {"loi vao", "lối vào", "cua vao", "entrance", "entry"}
CASHIER_KEYWORDS = {"cashier", "thu ngan", "quay thu ngan", "quầy thu ngân"}
COLD_KEYWORDS = {
    "tu mat",
    "tủ mát",
    "tu dong",
    "tủ đông",
    "chiller",
    "freezer",
    "refrigerated",
}

# (Tùy chọn) ép cứng danh sách Category lạnh
COLD_CATEGORIES = set(
    [
        # ví dụ: "Sữa chua", "Thịt đông lạnh"
    ]
)

# =========================
# HELPERS
# =========================
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


def normalize(s: str) -> str:
    if s is None:
        return ""
    s = s.strip().lower()
    return "".join(
        ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn"
    )


def has_any(text: str, keywords: set) -> bool:
    t = normalize(text)
    return any(k in t for k in keywords)


def load_shapes_from_ppt_mm(ppt_path: str) -> pd.DataFrame:
    # Đọc shapes có text, lưu cả slide_idx/shape_idx để giữ thứ tự ổn định khi ghi ngược
    EMU_PER_MM = 914400 / 25.4
    prs = Presentation(ppt_path)
    rows = []
    for s_idx, slide in enumerate(prs.slides):
        for sh_idx, shape in enumerate(slide.shapes):
            if not shape.has_text_frame:
                continue
            txt = (shape.text_frame.text or "").strip()
            if not txt:
                continue
            rows.append(
                {
                    "slide_idx": s_idx,
                    "shape_idx": sh_idx,
                    "shape_obj": shape,  # pointer để ghi ngược
                    "Category": txt,
                    "x": shape.left / EMU_PER_MM,
                    "y": shape.top / EMU_PER_MM,
                    "width": shape.width / EMU_PER_MM,
                    "height": shape.height / EMU_PER_MM,
                }
            )
    if not rows:
        raise RuntimeError("Không đọc được shape nào từ PPTX.")
    df = pd.DataFrame(rows)
    # cờ sơ bộ từ text (fallback khi CSV không có)
    df["is_entrance"] = df["Category"].apply(lambda s: has_any(s, ENTRANCE_KEYWORDS))
    df["is_cashier"] = df["Category"].apply(lambda s: has_any(s, CASHIER_KEYWORDS))
    df["is_refrigerated"] = df["Category"].apply(lambda s: has_any(s, COLD_KEYWORDS))
    return df, prs


def rect_center(row):
    return (row["x"] + row["width"] / 2.0, row["y"] + row["height"] / 2.0)


def manhattan(p, q):
    return abs(p[0] - q[0]) + abs(p[1] - q[1])


def euclid(p, q):
    return math.hypot(p[0] - q[0], p[1] - q[1])


def greedy_path_length(points, start, end, dist_fn=manhattan):
    if not points:
        return dist_fn(start, end)
    unvisited = points[:]
    cur = start
    total = 0.0
    while unvisited:
        nxt_i = min(range(len(unvisited)), key=lambda i: dist_fn(cur, unvisited[i]))
        nxt = unvisited.pop(nxt_i)
        total += dist_fn(cur, nxt)
        cur = nxt
    total += dist_fn(cur, end)
    return total


def parse_itemset(cell):
    # Lấy rule 1→1 từ PySpark (antecedent/consequent là chuỗi list)
    if pd.isna(cell):
        return None
    s = str(cell).strip()
    if not (s.startswith("[") and s.endswith("]")):
        return s if s else None
    try:
        val = ast.literal_eval(s)
        if isinstance(val, (list, tuple)) and len(val) == 1:
            return str(val[0])
        return None
    except Exception:
        inner = s.strip("[]").strip()
        parts = [p.strip(" '\"") for p in inner.split(",") if p.strip(" '\"")]
        return parts[0] if len(parts) == 1 else None


def estimate_cell_size_from_layout(df: pd.DataFrame) -> float:
    vals = []
    arr = df[["x", "y", "width", "height"]].values
    n = len(arr)
    for i in range(n):
        x0, y0, w0, h0 = arr[i]
        x0b, y0b = x0 + w0, y0 + h0
        for j in range(i + 1, n):
            x1, y1, w1, h1 = arr[j]
            x1b, y1b = x1 + w1, y1 + h1
            if not (y0b < y1 or y1b < y0):
                gx = max(0, x0 - x1b, x1 - x0b)
                if gx > 1:
                    vals.append(gx)
            if not (x0b < x1 or x1b < x0):
                gy = max(0, y0 - y1b, y1 - y0b)
                if gy > 1:
                    vals.append(gy)
    if vals:
        return max(1, int(min(vals)) // 2)
    min_dim = np.minimum(df["width"], df["height"])
    return (
        max(1, int(np.median(min_dim[min_dim > 0]) / 4)) if (min_dim > 0).any() else 5
    )


def rasterize_grid(df: pd.DataFrame, cell_size: float, padding_ratio: float = 0.06):
    cats = list(df["Category"].astype(str).unique())
    name2id = {c: i + 1 for i, c in enumerate(cats)}
    id2name = {v: k for k, v in name2id.items()}

    x0, y0 = df["x"].min(), df["y"].min()
    x1, y1 = (df["x"] + df["width"]).max(), (df["y"] + df["height"]).max()
    pad_x, pad_y = int((x1 - x0) * padding_ratio), int((y1 - y0) * padding_ratio)
    min_x, min_y = x0 - pad_x, y0 - pad_y
    max_x, max_y = x1 + pad_x, y1 + pad_y

    W = int(math.ceil((max_x - min_x) / cell_size))
    H = int(math.ceil((max_y - min_y) / cell_size))
    if W * H > 1e7:
        scale_factor = math.sqrt((W * H) / 1e7)
        cell_size *= scale_factor
        W = int(math.ceil((max_x - min_x) / cell_size))
        H = int(math.ceil((max_y - min_y) / cell_size))

    grid = np.zeros((H, W), dtype=np.int32)
    for _, r in df.iterrows():
        did = name2id[str(r["Category"])]
        gx0 = int(math.floor((r["x"] - min_x) / cell_size))
        gx1 = int(math.ceil((r["x"] + r["width"] - min_x) / cell_size))
        gy0 = int(math.floor((r["y"] - min_y) / cell_size))
        gy1 = int(math.ceil((r["y"] + r["height"] - min_y) / cell_size))
        grid[gy0:gy1, gx0:gx1] = did

    meta = {"min_x": min_x, "min_y": min_y, "cell_size": cell_size, "W": W, "H": H}
    return grid, name2id, id2name, meta


def visualize_pretty(df_layout: pd.DataFrame, out_png: str):
    cell = estimate_cell_size_from_layout(df_layout)
    grid, name2id, id2name, meta = rasterize_grid(
        df_layout, cell, padding_ratio=PADDING_RATIO
    )

    H, W = grid.shape
    unique_ids = np.unique(grid)
    max_id = int(unique_ids.max()) if len(unique_ids) > 0 else 0

    colors = ["#FFFFFF"] + [
        plt.cm.get_cmap("tab20", max(1, max_id))(i) for i in range(max_id)
    ]
    cmap = mcolors.ListedColormap(colors)
    bounds = list(range(0, max_id + 2))
    norm = mcolors.BoundaryNorm(bounds, cmap.N)

    fig_w = min(30, 18 * (W / max(1, H)))
    fig_h = min(30, 18)
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    ax.imshow(grid, cmap=cmap, norm=norm, interpolation="none")

    for did in np.unique(grid[grid > 0]):
        ys, xs = np.where(grid == did)
        if len(xs) == 0:
            continue
        cx, cy = np.mean(xs), np.mean(ys)
        name = id2name.get(did, f"ID {did}")
        region_color = cmap(norm(did))
        lum = (
            0.299 * region_color[0] + 0.587 * region_color[1] + 0.114 * region_color[2]
        )
        txt_color = "white" if lum < 0.5 else "black"
        w = xs.max() - xs.min() + 1
        h = ys.max() - ys.min() + 1
        rot = 90 if h > w * 1.6 and len(name) > 5 else 0
        fontsize = max(6, min(11, int(np.sqrt(w * h) / max(1, len(name)) * 4)))
        ax.text(
            cx,
            cy,
            name,
            va="center",
            ha="center",
            color=txt_color,
            fontsize=fontsize,
            rotation=rot,
            weight="bold",
        )

    ax.set_title(
        "Layout mới (preview dạng grid từ PPTX + flags từ CSV)",
        fontsize=14,
        weight="bold",
    )
    ax.grid(False)
    ax.tick_params(
        axis="both",
        which="both",
        bottom=False,
        left=False,
        labelbottom=False,
        labelleft=False,
    )
    plt.savefig(out_png, dpi=220, bbox_inches="tight")
    plt.close(fig)


def apply_layout_to_ppt(df_new: pd.DataFrame, prs_in: Presentation, out_pptx: str):
    """
    Ghi ngược: giữ nguyên vị trí shape, cập nhật text + màu theo Category mới.
    Chỉ sửa các shape bày hàng (không đụng Entrance/Cashier).
    Thứ tự shape = (slide_idx, shape_idx) — khớp với df_new đã đọc.
    """
    # Palette màu theo Category
    cats = list(
        df_new.loc[~(df_new["is_entrance"] | df_new["is_cashier"]), "Category"]
        .astype(str)
        .unique()
    )
    cmap = plt.cm.get_cmap("tab20", max(1, len(cats)))
    cat2rgb = {}
    for i, c in enumerate(cats):
        r, g, b, _ = cmap(i)
        cat2rgb[c] = (int(r * 255), int(g * 255), int(b * 255))

    # Lấy đúng danh sách shape placeable theo thứ tự đã đọc
    df_place = df_new[~(df_new["is_entrance"] | df_new["is_cashier"])].sort_values(
        ["slide_idx", "shape_idx"]
    )
    new_texts = df_place["Category"].tolist()

    # Tập shape objects tương ứng (cùng thứ tự)
    shapes_sorted = []
    for _, row in df_place.iterrows():
        shapes_sorted.append(row["shape_obj"])

    # Cập nhật
    for sh, new_cat in zip(shapes_sorted, new_texts):
        sh.text_frame.clear()
        p = sh.text_frame.paragraphs[0]
        run = p.add_run()
        run.text = str(new_cat)
        run.font.size = Pt(10)
        run.font.bold = True
        if new_cat in cat2rgb:
            r, g, b = cat2rgb[new_cat]
            sh.fill.solid()
            sh.fill.fore_color.rgb = RGBColor(r, g, b)
            if sh.line:
                sh.line.color.rgb = RGBColor(40, 40, 40)
                sh.line.width = Pt(0.75)

    prs_in.save(out_pptx)


# =========================
# LOAD PPTX + CSV FLAGS
# =========================
# 1) PPTX để lấy hình học + shape pointer
df_ppt, prs = load_shapes_from_ppt_mm(LAYOUT_PPTX)
df_ppt["cx"], df_ppt["cy"] = zip(*df_ppt.apply(rect_center, axis=1))

# 2) CSV để lấy flags (và Category nếu bạn muốn cập nhật lại)
try:
    df_csv = pd.read_csv(LAYOUT_CSV)
    # chuẩn hóa bool
    for col in ["is_refrigerated", "is_entrance", "is_cashier"]:
        if col in df_csv.columns:
            df_csv[col] = df_csv[col].apply(
                lambda v: str(v).strip().lower() in ["1", "true", "t", "yes", "y"]
            )
        else:
            df_csv[col] = False
    # join theo Category (giả định bạn đã chuẩn hóa tên trùng)
    # nếu có trùng tên nhiều lần, lấy OR (max) cho flags
    flag_agg = (
        df_csv.groupby("Category")[["is_refrigerated", "is_entrance", "is_cashier"]]
        .max()
        .reset_index()
    )
    df = df_ppt.merge(flag_agg, on="Category", how="left", suffixes=("", "_csv"))
    # nếu CSV có cờ thì dùng; nếu NaN thì fallback sang cờ từ text
    for col in ["is_refrigerated", "is_entrance", "is_cashier"]:
        if col + "_csv" in df.columns:
            df[col] = df[col + "_csv"].fillna(df[col]).astype(bool)
            df.drop(columns=[col + "_csv"], inplace=True)
except FileNotFoundError:
    # Không có CSV -> dùng flags suy luận từ text
    df = df_ppt.copy()

# Bắt buộc có Entrance & Cashier
if not df["is_entrance"].any() or not df["is_cashier"].any():
    raise RuntimeError(
        "Thiếu Entrance hoặc Cashier. Hãy đảm bảo CSV có cờ hoặc tên shape chứa keyword."
    )

entr_xy = (
    df.loc[df["is_entrance"], "cx"].iloc[0],
    df.loc[df["is_entrance"], "cy"].iloc[0],
)
cash_xy = (
    df.loc[df["is_cashier"], "cx"].iloc[0],
    df.loc[df["is_cashier"], "cy"].iloc[0],
)

# Chỉ các slot bày hàng (placeable)
slots = df[~(df["is_entrance"] | df["is_cashier"])].copy().reset_index(drop=True)
slots["slot_idx"] = np.arange(len(slots))
layout_cats = slots["Category"].astype(str).tolist()

# Xác định slot lạnh: ưu tiên CSV flags; nếu thiếu, suy luận theo keyword
slot_is_cold = slots["is_refrigerated"].astype(bool).tolist()

# =========================
# ASSOCIATION RULES (1→1)
# =========================
rules = pd.read_csv(ASSOC_RULES_CSV)
rules["_a"] = rules["antecedent"].apply(parse_itemset)
rules["_b"] = rules["consequent"].apply(parse_itemset)
pairs = rules.dropna(subset=["_a", "_b"]).copy()
pairs["weight"] = pairs["lift"] if "lift" in pairs.columns else pairs["confidence"]
pairs = pairs[["_a", "_b", "weight"]].rename(columns={"_a": "cat_a", "_b": "cat_b"})
pairs = pairs[pairs["cat_a"].isin(layout_cats) & pairs["cat_b"].isin(layout_cats)]
pairs = pairs[pairs["cat_a"] != pairs["cat_b"]]
pairs = pairs.groupby(["cat_a", "cat_b"], as_index=False)["weight"].mean()
pairs_list = [
    (a, b, w) for a, b, w in zip(pairs["cat_a"], pairs["cat_b"], pairs["weight"])
]

# =========================
# TRANSACTIONS × SKU  ->  BASKETS theo SDeptName
# =========================
tx = pd.read_csv(TRANSACTIONS_CSV, usecols=["Sku", "MergedId"])
sku = pd.read_csv(SKU_CSV, usecols=["Sku", CATEGORY_COL])
tx["Sku"] = tx["Sku"].astype(str)
sku["Sku"] = sku["Sku"].astype(str)
sku[CATEGORY_COL] = sku[CATEGORY_COL].astype(str)

tx = tx.merge(sku, on="Sku", how="left").dropna(subset=[CATEGORY_COL, "MergedId"])
baskets = (
    tx.groupby("MergedId")[CATEGORY_COL]
    .apply(lambda s: set(map(str, s.unique())))
    .tolist()
)

cat_counts = defaultdict(int)
for b in baskets:
    for c in b:
        cat_counts[c] += 1
n_baskets = max(1, len(baskets))
cat_support = {c: cat_counts.get(c, 0) / n_baskets for c in layout_cats}

# =========================
# OPTIMIZATION (SA với chi phí hiện tại)
# =========================
coords = np.array(list(zip(slots["cx"].values, slots["cy"].values)))
current_assign = {str(r["Category"]): i for i, r in slots.iterrows()}
cats_all = layout_cats[:]

# cold categories = category nào hiện đang ngồi slot lạnh HOẶC nằm trong COLD_CATEGORIES
cold_cats = set(str(r["Category"]) for _, r in slots.iterrows() if r["is_refrigerated"])
cold_cats |= set(COLD_CATEGORIES)

if len(cold_cats) > int(sum(slot_is_cold)):
    raise RuntimeError(
        f"Số category lạnh ({len(cold_cats)}) > số slot lạnh ({int(sum(slot_is_cold))}). Kiểm tra CSV hoặc giảm COLD_CATEGORIES."
    )

# precompute mean_dist cho scale
if len(coords) >= 2:
    rand_idx = np.random.choice(
        len(coords), size=(min(500, len(coords) * 2), 2), replace=True
    )
    MEAN_DIST = np.mean([euclid(coords[i], coords[j]) for i, j in rand_idx])
else:
    MEAN_DIST = 1.0


def objective(assign_map):
    # (1) kéo cặp lại gần
    pair_cost = 0.0
    for a, b, w in pairs_list:
        ia = assign_map.get(a)
        ib = assign_map.get(b)
        if ia is None or ib is None:
            continue
        pair_cost += w * euclid(coords[ia], coords[ib])
    pair_cost = pair_cost / max(1e-9, MEAN_DIST)
    # (2) kéo support cao gần Entrance
    ent_cost = 0.0
    for c in cats_all:
        s = (cat_support.get(c, 0.0)) ** GAMMA_SUPPORT
        ent_cost += s * euclid(entr_xy, coords[assign_map[c]])
    ent_cost = ent_cost / max(1e-9, MEAN_DIST * len(cats_all))
    return ALPHA_PAIR * pair_cost + BETA_ENTRANCE * ent_cost


def is_feasible(assign_map):
    used = set()
    for c in cats_all:
        si = assign_map[c]
        if slot_is_cold[si] and c not in cold_cats:
            return False
        if (not slot_is_cold[si]) and (c in cold_cats):
            return False
        if si in used:
            return False
        used.add(si)
    return True


def temperature(t):
    frac = t / max(1, SA_ITERS - 1)
    return SA_START_TEMP * (SA_END_TEMP / SA_START_TEMP) ** frac


assign = current_assign.copy()
curr_cost = objective(assign)
best_assign = assign.copy()
best_cost = curr_cost

for t in range(SA_ITERS):
    T = temperature(t)
    a, b = random.sample(cats_all, 2)
    ia, ib = assign[a], assign[b]
    # ràng buộc lạnh
    if slot_is_cold[ia] != (b in cold_cats):
        continue
    if slot_is_cold[ib] != (a in cold_cats):
        continue
    # thử swap
    assign[a], assign[b] = ib, ia
    if not is_feasible(assign):
        assign[a], assign[b] = ia, ib
        continue
    new_cost = objective(assign)
    delta = new_cost - curr_cost
    if (delta < 0) or (random.random() < math.exp(-delta / max(1e-9, T))):
        # chấp nhận trạng thái mới
        curr_cost = new_cost
        if new_cost < best_cost:
            best_cost = new_cost
            best_assign = assign.copy()
    else:
        # revert
        assign[a], assign[b] = ia, ib

# =========================
# XUẤT CSV + PREVIEW + GHI PPTX
# =========================
df_new = df.copy()
# cập nhật Category cho các slot bày hàng theo best_assign (giữ nguyên Entrance/Cashier)
slotidx_to_cat = {best_assign[c]: c for c in cats_all}
for i, row in slots.iterrows():
    new_cat = slotidx_to_cat.get(i, row["Category"])
    df_new.at[slots.index[i], "Category"] = new_cat

# Lưu CSV (để tái dùng)
out_cols = [
    "Category",
    "x",
    "y",
    "width",
    "height",
    "is_refrigerated",
    "is_entrance",
    "is_cashier",
    "slide_idx",
    "shape_idx",
    "cx",
    "cy",
    "shape_obj",
]
df_new.to_csv(OUTPUT_LAYOUT_CSV, index=False, encoding="utf-8-sig")

# Preview đẹp
visualize_pretty(df_new, OUTPUT_PREVIEW_PNG)

# Ghi ngược PPTX
apply_layout_to_ppt(df_new, prs, OUTPUT_PPTX)

# =========================
# BENCHMARK ĐƯỜNG ĐI (midpoint-hopping, Manhattan)
# =========================
orig_cat2xy = {
    str(r["Category"]): (r["cx"], r["cy"])
    for _, r in df.iterrows()
    if not (r["is_entrance"] or r["is_cashier"])
}
new_cat2xy = {
    str(r["Category"]): (r["cx"], r["cy"])
    for _, r in df_new.iterrows()
    if not (r["is_entrance"] or r["is_cashier"])
}


def avg_path_length(cat2xy):
    sample_b = baskets if len(baskets) <= 500 else random.sample(baskets, 500)
    lens = []
    for b in sample_b:
        pts = [cat2xy[c] for c in b if c in cat2xy]
        L = greedy_path_length(pts, entr_xy, cash_xy, dist_fn=manhattan)
        lens.append(L)
    return np.mean(lens) if lens else float("nan")


base_L = avg_path_length(orig_cat2xy)
new_L = avg_path_length(new_cat2xy)
improve = (
    (base_L - new_L) / base_L * 100
    if (base_L and not math.isnan(base_L) and base_L > 0)
    else float("nan")
)

print(f"[DONE] Xuất:\n- {OUTPUT_LAYOUT_CSV}\n- {OUTPUT_PREVIEW_PNG}\n- {OUTPUT_PPTX}")
print(f"Độ dài đường đi TB (Manhattan, midpoint-hopping):")
print(f"- Layout cũ:  {base_L:.2f}")
print(f"- Layout mới: {new_L:.2f}")
print(f"→ Cải thiện:  {improve:.2f}%")

C:\Users\admin\AppData\Local\Temp\ipykernel_23444\2175460560.py:198: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = ['#FFFFFF'] + [plt.cm.get_cmap('tab20', max(1, max_id))(i) for i in range(max_id)]


[DONE] Xuất:
- C:\Users\admin\Desktop\retail-recommendation-system\data\layout\output\layout_new.csv
- C:\Users\admin\Desktop\retail-recommendation-system\data\layout\output\layout_new.png
- C:\Users\admin\Desktop\retail-recommendation-system\data\layout\output\layout_new.pptx
Độ dài đường đi TB (Manhattan, midpoint-hopping):
- Layout cũ:  325.49
- Layout mới: 315.76
→ Cải thiện:  2.99%


C:\Users\admin\AppData\Local\Temp\ipykernel_23444\2175460560.py:234: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab20', max(1, len(cats)))
